# Training

In this chapter we will use all the stuff we learned to finally train a neural network 🤖 using neuroimaging data 🧠

So, get ready for your new favorite hobby 🤓

<p>
<img src="https://programmerhumor.io/wp-content/uploads/2023/05/programmerhumor-io-python-memes-programming-memes-d917ae7c7cb4095-758x495.jpg" width=500/>
<figcaption>Taken from <a href="https://programmerhumor.io/python-memes/its-been-a-decade/">https://programmerhumor.io/python-memes/its-been-a-decade/</a></figcaption>
</p>

## 1. What is training?

During model training (the sentence continues here 👇)

```python
...
neural_net = NeuralNet()          # the neural net              
for nifti, age in dl:             # is provided with training input
  nifti_age = neural_net(nifti)   # and the neural nets output
  error = (age - nifti_age) ** 2  # is compared to the desired output
  error.backward()                # and the net is adjusted (using Autograd 🦮)
```
Wow, this code is already pretty close to our goal (3 more lines to make it work). We are only missing one final ingredient: The optimizer!

## 2. `torch.nn.optim`

The module `torch.nn.optim` provides a collection of optimizers which handle the adjustment of (neural net-) parameters.

First the optimizer is given the parameters it should update

```python
...   
neural_net = NeuralNet()
optimizer = torch.optim.Adam(neural_net.parameters())  # creating the optimizer
```
then it can be applied in the above code by adding one line at the beginning and one at the end of the training loop

```python          
for nifti, age in dl:
  optimizer.zero_grad()           # resetting the parameters gradients to zero
  nifti_age = neural_net(nifti)
  error = (age - nifti_age) ** 2
  error.backward()
  optimizer.step()                # adjusting the parameters (using Autograd 🦮)
```
Those are the three missing lines which make the above code actually work.

The two most common ways to tinker with the optimizer are by
1. changing the learning rate to e.g. `1e-2` via `torch.optim.Adam(neural_net.parameters(), lr=1e-2)`
2. changing the optimizer e.g. using `torch.optim.SGD(...)` instead of `torch.optim.Adam(...)`

`torch.optim.SGD(...)` - Stochastic Gradient Descent - is simply changing the parameters by $lr * -\nabla$ (re-read 5_PyTorch, 2. Autograd 🦮)

`torch.optim.Adam(...)` is typically **your optimizer of choice** (doing [a bit more](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html))  working good with `lr` between `1e-5` and `1e-1`


# Exercise

## 🚨 Warning 🚨

This Notebook builds on 1_Introduction and the exercise of 2_Data_Exploration and 4_Preprocessing.

You have to run these Notebooks (if you didn't already) and mount your Google Drive to this Notebook via
```python
from google.colab import drive
drive.mount('/content/drive')
```
then you are ready to go!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Use the exercise solution of 6_Convolutional_Neural_Net and add the training loop shown in chapter 2 `torch.nn.optim`!

In [ ]:
import torch
import numpy as np
import nibabel as nib
import torch.nn.functional as F

class NiftiDataset(torch.utils.data.Dataset):
    def __init__(self, filepaths, age_array, size=(128, 128, 128)):
        self.niftis = [self.load_nifti(fpath, size) for fpath in filepaths]
        self.ages = torch.from_numpy(age_array)

    def __len__(self):
        return len(self.niftis)

    def __getitem__(self, idx):
        return self.niftis[idx], self.ages[idx].float()

    @staticmethod
    def load_nifti(filepath, size):
      img = nib.load(filepath)
      img = nib.as_closest_canonical(img)
      x = img.get_fdata(dtype=np.float32)
      x = torch.from_numpy(x)
      x = x[None, None]  # added 2 dimensions using [None, None] to make it 5D
      x = F.interpolate(x, size=size)  # resized it to (1, 1, 128, 128, 128)
      return x[0]  # Removed one dimension using [0] because DataLoader will add one later

In [ ]:
import torch.nn as nn

class NeuralNet(nn.Module):
  def __init__(self, kernel_size=3):
    super().__init__()
    self.conv1 = nn.Conv3d(1, 4, kernel_size=kernel_size)
    self.pool1 = nn.MaxPool3d(kernel_size=kernel_size)
    self.conv2 = nn.Conv3d(4, 16, kernel_size=kernel_size)
    self.pool2 = nn.MaxPool3d(kernel_size=kernel_size)
    self.conv3 = nn.Conv3d(16, 32, kernel_size=kernel_size)
    self.pool3 = nn.MaxPool3d(kernel_size=kernel_size)
    self.conv4 = nn.Conv3d(32, 1, kernel_size=kernel_size)

  def forward(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.conv3(x)
    x = self.pool3(x)
    x = self.conv4(x)
    return x

In [ ]:
import pandas as pd
PATH = '/content/drive/MyDrive/openneuro'

df = pd.read_csv(f'{PATH}/dataframe_after_preprocessing.csv', index_col=0)
ds = NiftiDataset(df.zscore_filepath, age_array=df.age.values)
dl = torch.utils.data.DataLoader(ds, batch_size=1, shuffle=True)

2. Print `error`, `nifti_age` and `age` inside the training loop!

In [ ]:
neural_net = NeuralNet(kernel_size=3)
optimizer = torch.optim.Adam(neural_net.parameters(), lr=1e-3)

for nifti, age in dl:
  optimizer.zero_grad()
  nifti_age = neural_net(nifti)
  error = (age - nifti_age) ** 2
  print(error, age, nifti_age)
  error.backward()
  optimizer.step()

tensor([[[[[679.8942]]]]], grad_fn=<PowBackward0>) tensor([26.]) tensor([[[[[-0.0748]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[456.3232]]]]], grad_fn=<PowBackward0>) tensor([22.]) tensor([[[[[0.6383]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[383.3004]]]]], grad_fn=<PowBackward0>) tensor([21.]) tensor([[[[[1.4219]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[563.4132]]]]], grad_fn=<PowBackward0>) tensor([26.]) tensor([[[[[2.2637]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[229.5711]]]]], grad_fn=<PowBackward0>) tensor([19.]) tensor([[[[[3.8484]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[208.2583]]]]], grad_fn=<PowBackward0>) tensor([20.]) tensor([[[[[5.5688]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[258.8898]]]]], grad_fn=<PowBackward0>) tensor([24.]) tensor([[[[[7.9099]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[200.8020]]]]], grad_fn=<PowBackward0>) tensor([24.]) tensor([[[[[9.8295]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[227.9518]]]

3. Use the exercise solution of 5_PyTorch to **create separate training and validation DataLoaders**.

In [ ]:
valid_pids = [f'sub-01', 'sub-02', 'sub-03']

train_df = df[~df.index.isin(valid_pids)]
valid_df = df[df.index.isin(valid_pids)]

train_ds = NiftiDataset(train_df.t1w_filepath, age_array=train_df.age.values)
valid_ds = NiftiDataset(valid_df.t1w_filepath, age_array=valid_df.age.values)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=1, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=1, shuffle=False)

4. Run the training loop (with print statements) **5 times** - using an additional `for` loop - with the **training DataLoader**!

In [ ]:
neural_net = NeuralNet(kernel_size=3)
optimizer = torch.optim.Adam(neural_net.parameters(), lr=1e-3)


for epoch in range(5):
  for nifti, age in train_dl:
    optimizer.zero_grad()
    nifti_age = neural_net(nifti)
    error = (age - nifti_age) ** 2
    print(error, age, nifti_age)
    error.backward()
    optimizer.step()

tensor([[[[[1016.8278]]]]], grad_fn=<PowBackward0>) tensor([24.]) tensor([[[[[55.8877]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[12580.0459]]]]], grad_fn=<PowBackward0>) tensor([22.]) tensor([[[[[-90.1608]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[1967.8070]]]]], grad_fn=<PowBackward0>) tensor([26.]) tensor([[[[[-18.3600]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[28.9679]]]]], grad_fn=<PowBackward0>) tensor([20.]) tensor([[[[[25.3822]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[4496.9248]]]]], grad_fn=<PowBackward0>) tensor([21.]) tensor([[[[[88.0591]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[4871.1641]]]]], grad_fn=<PowBackward0>) tensor([22.]) tensor([[[[[91.7937]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[0.4453]]]]], grad_fn=<PowBackward0>) tensor([30.]) tensor([[[[[29.3327]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[366.9161]]]]], grad_fn=<PowBackward0>) tensor([21.]) tensor([[[[[1.8449]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[

5. Use the **trained `neural_net`** to run the loop (with print statements) **one time** with the **validation DataLoader**. **Delete/Uncomment all lines which could adjust the neural nets parameters** as we do not want to train during validation.

In [ ]:
  for nifti, age in valid_dl:
    # optimizer.zero_grad()
    nifti_age = neural_net(nifti)
    error = (age - nifti_age) ** 2
    print(error, age, nifti_age)
    # error.backward()
    # optimizer.step()

tensor([[[[[52.1319]]]]], grad_fn=<PowBackward0>) tensor([26.]) tensor([[[[[18.7798]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[4.8318]]]]], grad_fn=<PowBackward0>) tensor([24.]) tensor([[[[[26.1981]]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[[25.4240]]]]], grad_fn=<PowBackward0>) tensor([27.]) tensor([[[[[21.9578]]]]], grad_fn=<ConvolutionBackward0>)
